In [1]:
import cv2
from ultralytics import YOLO

In [1]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")
# Load the image
image_path = "image6.jpg"
image = cv2.imread(image_path)
height, width, channels = image.shape

# Manually noted coordinates of the A4 paper bounding box
# a4_x1, a4_y1 = 100, 100  # Replace with actual values
# a4_x2, a4_y2 = 600, 800  # Replace with actual values

# Calculate the width and height in pixels
# a4_width_pixels = width
# a4_height_pixels = height

# Real-world dimensions of the A4 paper (in cm)
# A4_WIDTH_CM = 21.0
# A4_HEIGHT_CM = 29.6

# Calculate the conversion factors
# cm_per_pixel_width = A4_WIDTH_CM / a4_width_pixels
# cm_per_pixel_height = A4_HEIGHT_CM / a4_height_pixels


# Run YOLOv8 detection on the image
results = model.predict(image)

# Extract the detections from the results
detections = results[0].boxes

# Known dimensions of the smartphone in centimeters
SMARTPHONE_WIDTH_CM = 7.0
SMARTPHONE_HEIGHT_CM = 15.0

# Smartphone screen resolution in pixels (example values, you need to replace these with actual values)
SMARTPHONE_WIDTH_PIXELS = 2298
SMARTPHONE_HEIGHT_PIXELS = 1045

# Loop through the detections
for detection in detections:
    # Get the bounding box coordinates
    x1, y1, x2, y2 = map(int, detection.xyxy[0])

    # Calculate the width and height of the bounding box in pixels
    width_pixels = x2 - x1
    height_pixels = y2 - y1

    # Convert the width and height to centimeters using the smartphone screen dimensions
    width_cm = (width_pixels / SMARTPHONE_WIDTH_PIXELS) * SMARTPHONE_WIDTH_CM
    height_cm = (height_pixels / SMARTPHONE_HEIGHT_PIXELS) * SMARTPHONE_HEIGHT_CM

    # Draw the bounding box on the image
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display the width and height in cm on the image
    label = f"W: {width_cm:.2f} cm, H: {height_cm:.2f} cm"
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the annotated image
cv2.imshow("YOLOv8 Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the annotated image
cv2.imwrite('Detected_objects_with_dimensions2.jpg', image)


0: 288x640 1 cell phone, 1 book, 78.0ms
Speed: 5.8ms preprocess, 78.0ms inference, 5.6ms postprocess per image at shape (1, 3, 288, 640)


True

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Define the class ID for a phone
PHONE_CLASS_ID = 67

# Manually noted coordinates of the A4 paper bounding box
# x1, y1 = 100, 100  # Replace with actual values
# x2, y2 = 600, 800  # Replace with actual values

# Calculate the width and height in pixels
# a4_width_pixels = x2 - x1
# a4_height_pixels = y2 - y1

# Real-world dimensions of the A4 paper (in cm)
# A4_WIDTH_CM = 21.0
# A4_HEIGHT_CM = 29.6

# # Calculate the conversion factors
# cm_per_pixel_width = A4_WIDTH_CM / a4_width_pixels
# cm_per_pixel_height = A4_HEIGHT_CM / a4_height_pixels

# Known dimensions of the smartphone in centimeters
SMARTPHONE_WIDTH_CM = 7.0
SMARTPHONE_HEIGHT_CM = 15.0

# Smartphone screen resolution in pixels 
SMARTPHONE_WIDTH_PIXELS = 585
SMARTPHONE_HEIGHT_PIXELS = 275

# Open the video file
cap = cv2.VideoCapture('video-smartphone.mp4')

# Function to convert bounding box dimensions from pixels to centimeters
def convert_pixels_to_cm(width_pixels, height_pixels, ref_width_pixels, ref_height_pixels):
    pixel_to_cm_width = SMARTPHONE_WIDTH_CM / ref_width_pixels
    pixel_to_cm_height = SMARTPHONE_HEIGHT_CM / ref_height_pixels
    width_cm = width_pixels * pixel_to_cm_width
    height_cm = height_pixels * pixel_to_cm_height
    return width_cm, height_cm

while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    
    if not success:
        break

    # Run YOLOv8 detection on the frame
    results = model.track(frame, persist=True)

    # Extract the detections from the results
    detections = results[0].boxes.xywh.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    # Loop through the detections
    for detection, class_id in zip(detections, classes):
        if class_id == PHONE_CLASS_ID:
            cx, cy, w, h = detection

            # Convert the center coordinates to top-left corner
            x1 = int(cx - w / 2)
            y1 = int(cy - h / 2)
            x2 = int(cx + w / 2)
            y2 = int(cy + h / 2)

            # Convert the width and height to centimeters
            width_cm, height_cm = convert_pixels_to_cm(w, h, SMARTPHONE_WIDTH_PIXELS, SMARTPHONE_HEIGHT_PIXELS)

            # Draw the bounding box on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Display the width and height in cm on the frame
            label = f"W: {width_cm:.2f} cm, H: {height_cm:.2f} cm"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the annotated frame
    cv2.imshow("YOLOv8 Tracking", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 cell phone, 87.7ms
Speed: 4.5ms preprocess, 87.7ms inference, 6.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cell phone, 79.8ms
Speed: 2.4ms preprocess, 79.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cell phone, 76.6ms
Speed: 1.6ms preprocess, 76.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cell phone, 78.7ms
Speed: 1.9ms preprocess, 78.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cell phone, 77.0ms
Speed: 2.7ms preprocess, 77.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cell phone, 69.3ms
Speed: 1.4ms preprocess, 69.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cell phone, 67.7ms
Speed: 1.7ms preprocess, 67.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cell phone, 66.2ms
Speed: 2.9ms preprocess, 66.2ms inference, 0.4ms postproc